In [1]:
from lively import Solver, Translation, Rotation,Transform,SmoothnessMacroObjective, CollisionAvoidanceObjective, JointLimitsObjective, BoxShape, CollisionSettingInfo, ScalarRange
from lxml import etree

# Constructed CollisionAvoidance Objective ,BoxShape and configured Collision Setting
box = BoxShape(name="Table",frame="world",physical=True,x=2,y=1,z=1.2,translation = Translation(x=1.0, y =2.0, z=3.0),
rotation = Rotation(x=0.0,y=0.0,z=0.0,w=1.0))
collision = CollisionSettingInfo(d_max = 0.1, r = 0.0, a_max = 2.0, time_budget = 100, timed = False)

# Read the xml file into a string
xml_file = '../../tests/panda.xml'
tree = etree.parse(xml_file)
xml_string = etree.tostring(tree).decode()

#print(xml_string)
# Instantiate a new solver
solver = Solver(
  urdf=xml_string, # Full urdf as a string
  objectives={
      # An example objective (smoothness macro)
      "smoothness":SmoothnessMacroObjective(name="MySmoothnessObjective",weight=5,joints=True,origin=False,links=True),
      "collision": CollisionAvoidanceObjective(name="MyCollisionAvoidanceObjective", weight=5),
      "jointLimit": JointLimitsObjective(name="MyJointLimitObjective", weight=5)
      
  },
  root_bounds=[
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0), # Translational, (x, y, z)
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0)  # Rotational, (r, p, y)
  ],
  shapes=[
      box
  ],
  collision_settings = collision

)

# Run solve to get a solved state
state = solver.solve(goals= {"MyGoal1": Translation(x=1.0, y =2.0, z=3.0)},weights = {},time = 0.0)
# Log the initial state
print(state.origin.as_dicts())
print(state.joints)
print(state.frames["panda_link0"])
print(state.proximity)

({'x': 0.0, 'z': 0.0, 'y': 0.0}, {'x': 0.0, 'w': 1.0, 'y': 0.0, 'z': 0.0})
{'panda_joint1': -1.7584339786062264e-07, 'panda_joint3': -1.7509688830476996e-07, 'panda_finger_joint2': 0.029211290070767667, 'panda_joint5': -3.8995041672062055e-08, 'panda_joint2': -4.545444325457867e-06, 'panda_joint4': -1.5708393176592177, 'panda_joint6': 1.8675023747631656, 'panda_finger_joint1': 0.029211290070767667, 'panda_joint7': -0.00014566664876653805}
TransformInfo: world:{translation: {x:0.0, y:0.0, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}, local: {translation: {x:0.0, y:0.0, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}
[ProximityInfo: {shape1: "panda_link1", shape2: "panda_link2", distance: -0.04999999999998839, points: Some(([-2.675556898748609e-7, 1.249786190593223e-13, 0.39299999999939733], [-5.062541290606117e-8, 9.06965474944954e-14, 0.34299999999987957])), physical: true, loss: 1.0499999999999883, average_distance: Some(1.0)}, ProximityInfo: {shape1: "panda_link3", shape2: "panda_